# <p style="text-align: center;">MIS 285N: Big Data and Distributed Programming</p>
# <p style="text-align: center;">Project - 1 : Apache Spark</p>
## <p style="text-align: center;">Instructor: Dr. Ramesh Yerraballi</p>
## <p style="text-align: center;">Due: Tuesday, September 14th submitted via Canvas by 11:59 pm</p>

Your work should be written in a **Jupyter notebook**.   

Also, please make sure your code runs in your notebook before submitting.

**Note:**

This project is based on Map-Reduce Framework. In these you will get to work with Spark and will get to know how 
does spark work, what functionalities does spark provide, what does map-reduce framework do and why is it useful. 

In this project you will be implementing a basic song recommender system. You will be given a dataset where there are multiple csv files. These csv files have data corresponding to song play count and song information.

The data you will use is provided in a zip file along with this notebook. The __msd.zip__ archive contains:
1. **'kaggle_visible_evaluation_triplets.txt'**. We will be using the visible part of the testing data to understand the working on Apache Spark.  The user's listening history is provided as: (user, song, play count).  
2. In **'kaggle_songs.txt'** file, each song is marked using an index for easier representation of songs.  
3. And **'kaggle_users.txt'** file is the canonical list of user identifiers.
4. Take **'MSDChallengeGettingstarted.pdf'** as your reference.



### **What to turn in?**  

A zip folder which will have:
1. Jupyter Notebook
2. A brief report in PDF format on what features you used for recommendation. And a brief explanation of flow of your code. For example,  what RDD does what or, why it was created.
3. datasets folder with the csv files you are using in your notebook.
4. Notebook should use relative path to the csv files in datasets folder.
5. Name of the zip folder - `<your_name>_<your_partner_name>.zip`

This project consists of 4 questions:  

1. Create an RDD with _kaggle_visible_evaluation_triplets.txt_ and replace the song name with the song index from _kaggle_songs.txt_. Identify the number of songs that do not have any rating. 
2. Generate song ratings based on the song play count as a normalized score between 0 and 1. 
3. Identify the popular song based on this rating and recommend songs to user, given user id based on the algorithm used in Movie recommender system from class. 
4. Using Cosine similarity function, identify pair-wise similarity between each pair of users and generate the top 5 most similar users without an overlap in users. 

The above list is the high-level idea about the questions. 

In [4]:
### Starter code ####
import findspark
findspark.init()   #'/users/domitillechambon/spark-3.3.0-bin-hadoop3/'
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local[*]").setAppName("Songs")
sc = SparkContext.getOrCreate(conf = conf)
#### These lines are to tell jupyter where to find Apache Spark ####

In [5]:
#sc.stop()

In [6]:
## Read triplet file into RDD
triplet_rdd = sc.textFile(r"kaggle_visible_evaluation_triplets.txt") \
    .map(lambda line: line.split("\t")) 

## Step 1: 
Replace song name with song index and identify the number of songs without user history

In [7]:
# Reading in songs file into RDD
song_rdd = sc.textFile(r"kaggle_songs.txt") \
    .map(lambda line: line.split(" ")) 

In [8]:
## SWITCHING SONG NAME FOR ITS INDEX
# Reformatting the triplet and song RDDs
tempTriplet = triplet_rdd.map(lambda x: (x[1], [x[0], x[2]]))
tempSong = song_rdd.map(lambda x: (x[0], x[1]))

# Merging the reformatted RDDs
tempMerged = tempTriplet.join(tempSong)

# Reformatting the merge
updatedTriplet = tempMerged.map(lambda x: (x[1][0][0], x[1][1], x[1][0][1]))

In [9]:
# FINDING SONGS THAT HAVE NO RATINGS
# Creating distinct list of songs users have interacted with and list of all songs
userSongs = updatedTriplet.map(lambda x: x[1]).distinct()
allSongs = song_rdd.map(lambda x: x[1])

# Calculating number of songs with no ratings
songsNoRatings = allSongs.subtract(userSongs).count()
print(songsNoRatings, "songs don't have a rating.")

223007 songs don't have a rating.


## Step 2:
Generate song ratings based on the play_count. For example, if (song_1, 5; song_2, 10; song_3, 5) i.e., song_1 is played 5 times, song_2 is played 10 times and song_3 is played 5 times, the normalized rating score should be 0.25, 0.5 and 0.25 respectively. 
Similarly, generate the rating for all the songs. You may notice that based on all songs, the rating is almost always very low. So, think of the best way to convert song count to ratings. (Hint: Try generating ratings based on each user's song play history)

In [10]:
## MAKE UPDATED TRIPLET INTO A DICTIONARY WITH LIST OF LISTS
# Formatting UpdatedTriplet to be like a dictionary
utDict = updatedTriplet.map(lambda x: (x[0], [x[1], x[2]]))

# Grouped by users
groupedTriplet = utDict.groupByKey().map(lambda x: (x[0], list(x[1])))

In [11]:
## CALCULATE RATINGS
# Function to sum song plays for each user
def sumPlayCount(line):
    userPlays = line[0], list(map(lambda x: int(x[1]), line[1]))
    tempSum = sum(userPlays[1])
    sumRDD = line[0], list(map(lambda x: [x[0], int(x[1]), tempSum], line[1]))
    return sumRDD

# Function to calculate rating for each song for each user
def rating(line):
    userPlaySum = line[0], list(map(lambda x: [x[0], round(x[1] / x[2], 4)], line[1]))
    return userPlaySum

tripletSum = groupedTriplet.map(lambda x : sumPlayCount(x))
tripletRating = tripletSum.map(lambda x : rating(x))

tripletRating.take(5)

[('e4332e11f4df6dd26673bb6b085e9a2bbdc9b8a5',
  [['25150', 0.037],
   ['307202', 0.0185],
   ['212702', 0.0185],
   ['354593', 0.0185],
   ['292950', 0.0185],
   ['248176', 0.0185],
   ['294419', 0.0185],
   ['384072', 0.0926],
   ['248603', 0.0185],
   ['220482', 0.0185],
   ['126840', 0.0185],
   ['49781', 0.0185],
   ['191217', 0.0185],
   ['123630', 0.0926],
   ['319911', 0.0185],
   ['115162', 0.1852],
   ['327432', 0.0185],
   ['126627', 0.0185],
   ['39842', 0.0185],
   ['177574', 0.0185],
   ['334187', 0.0926],
   ['192716', 0.0185],
   ['30643', 0.0185],
   ['37133', 0.0185],
   ['362447', 0.0185],
   ['44743', 0.0185],
   ['243307', 0.0926],
   ['295585', 0.0185]]),
 ('f6e34f0a68d5ea1344511e33486f956de361db78',
  [['25150', 0.0046],
   ['202995', 0.0091],
   ['313367', 0.0776],
   ['195624', 0.0548],
   ['63758', 0.0548],
   ['94613', 0.0091],
   ['215039', 0.0046],
   ['137041', 0.0228],
   ['211253', 0.0228],
   ['190566', 0.0228],
   ['70235', 0.0685],
   ['516', 0.0228],


## Step 3: 
For a given user_id (choose one by yourselves), rating, recommend 5 other songs from the list. One way to do this is based on another user who liked the same song liked by this user with rating more than the given rating and recommend the 5 songs based on the matched user's rating. 

In [12]:
## CHOOSE USER AND FIND AVERAGE RATING OF SONGS THEY LISTEN TO
# Calculates average rating of songs for specified user
def avgRating(user):
    narrowRatings = list(map(lambda x: x[1], user[1]))
    averageRating = sum(narrowRatings) / len(narrowRatings)
    avgRDD = user[0], list(map(lambda x: round(averageRating, 4), user[1]))
    return avgRDD

# Calculates max rating in a specified user's song list
def maxRating(user):
    narrowRatings = list(map(lambda x: x[1], user[1]))
    maxRating = max(narrowRatings)
    maxRDD = user[0], list(filter(lambda x: x[1] == maxRating, user[1]))
    return maxRDD

# User we selected
selectedUser = "e4332e11f4df6dd26673bb6b085e9a2bbdc9b8a5"

# Rating user wants song to be above to listen to (average rating of songs they've listened to
userRating = tripletRating.filter(lambda x: x[0] == selectedUser) \
                        .map(lambda x: avgRating(x)) \
                        .map(lambda x: x[1][0]).collect()  

# User's most listened to song/song with highest rating
topSong = tripletRating.filter(lambda x: x[0] == selectedUser) \
                        .map(lambda x: maxRating(x)) \
                        .map(lambda x: x[1][0][0]).collect()

In [13]:
## CREATE RECOMMENDATIONS FOR USER
# Pulls all users who have listened to our select user's top song and the rating is ≥ user's preferred rating
usersWithTopSong = tripletRating.filter(lambda x: x[0] != selectedUser) \
                                .filter(lambda x: x[1][0][0] == topSong[0]) \
                                .filter(lambda x: x[1][0][1] >= userRating[0]) \

# Pull a list of all potential songs and ratings without user into list
tempListPotentialSongs = usersWithTopSong.map(lambda x: list(map(lambda y: [y[0], y[1]], x[1]))) \
                                        .flatMap(lambda x: x)

# Remove the user's top song from list and removes songs less than rating threshold
listPotentialSongs = tempListPotentialSongs.filter(lambda x: x[0] != topSong[0]) \
                                            .filter(lambda x: x[1] >= userRating[0])

# Create distinct list of song names with a list of ratings for that song if there are duplicates
temp = listPotentialSongs.groupByKey().map(lambda x: [x[0], list(x[1])])

# Average ratings of songs with more than 1
distinctSongsWithRating = temp.map(lambda x: [x[0], round(sum(x[1]) / len(x[1]), 4)])

# Remove songs with ratings less than the rating threshold
songsAboveRating = distinctSongsWithRating.filter(lambda x: x[1] >= userRating[0]).collect()

# Create list of songs the user has already listened to
userSongs = tripletRating.filter(lambda x: x[0] == selectedUser) \
                            .map(lambda x: list(map(lambda y: y[0], x[1]))).collect()

# Remove songs the user has already listened to from the potential recommendation list
for line in songsAboveRating:
    for song in userSongs:
        if line[0] == song:
            songsAboveRating.remove(line)

# Sort the final recommendations list in order of highest song rating to lowest            
sortedRecs = sorted(songsAboveRating, key= lambda rating: rating[1], reverse= True)
 
# Create empty recommendations list
recommendations = []    

# Add the top 5 songs from the sortedRecs list
for songs in sortedRecs[0:5]:
    recommendations.append(songs[0])

# Print the final recommendation
print("The recommendations for", selectedUser, "are:", recommendations)

The recommendations for e4332e11f4df6dd26673bb6b085e9a2bbdc9b8a5 are: ['334102', '132587', '142602', '225139', '29460']


## Step 4: 
1. Compute cosine similarity between all pairs of users. 
2. Sort the similarity score and print the top-5 similar users. 
3. If the top-5 user set has an user appearing more than once, ignore that pair and take the next best pair from the sorted list. 
4. For a given user_id, identify the top-5 similar users and hence song recommendations from other user's list. 

In [14]:
## COMPUTING COSINE SIMILARITY BETWEEN ALL PAIRS OF USERS. SORT THE SIMILARITY SCORE AND PRINT THE TOP-5 SIMILAR USERS. REMOVE USERS THAT APPEAR MORE THAN ONCE
import math
from operator import itemgetter

# RDD with user and list of songs & taking first 275 users
userSongs_rdd = groupedTriplet.map(lambda x: [x[0], list(map(lambda y: y[0], x[1]))])
songsUser = userSongs_rdd.map(lambda x: x).take(275)

# List of songs per user without user for first 275 users
songsWithoutUser = userSongs_rdd.map(lambda x: x[1]).take(275)

# List of unique songs that were listened to
unique_songs = sorted(list({ song 
                    for user in songsWithoutUser
                    for song in user}))

# Assign users to related index that populates later
userSongDict = {}
count = 0
for users in songsUser:
    userSongDict[count] = users[0]
    count += 1

# Functions
def dot(v,w):
    return sum(v_i * w_i
              for v_i, w_i in zip(v,w))

def cosine_similarity(v,w):
    return dot(v, w) / math.sqrt(dot(v, v) * dot(w, w))

def makeUserSongVector(userSongs):
    return [1 if song in userSongs else 0
            for song in unique_songs]

def mostSimilarUsersTo(userID):
    pairs = [(otherUser, similarity)
            for otherUser, similarity in 
                enumerate(userSimilarities[userID])
            if userID != otherUser and similarity > 0]
    
    return sorted(pairs, key= lambda pair: pair[1], reverse= True)

# Matrix of 1s and 0s
userSongMatrix = list(map(makeUserSongVector, songsWithoutUser))

# Calculate cosine similarity in Matrix format
userSimilarities = [[cosine_similarity(song_vector_i, song_vector_j)
                    for song_vector_j in userSongMatrix]
                    for song_vector_i in userSongMatrix]

# Empty list that will populate with the pairs and their cosine similarity
similarityPairs = []
for index, row in enumerate(userSimilarities):
    for indexCol, colValue in enumerate(row):
        temp = []
        temp.append(index)
        temp.append(indexCol)
        temp.append(colValue)
        similarityPairs.append(temp)

# Remove pairs that have cosine similarities equal to 1 and 0; switch index values for the user IDs
for pair in similarityPairs:
    if pair[2] == 1.0 or pair[2] == 0.0:
        similarityPairs.remove(pair)
    for user in userSongDict:
        if pair[0] == user:
            pair[0] = userSongDict[user]
        elif pair[1] == user:
            pair[1] = userSongDict[user]

# Sort pairs from highest similarity score to lowest   
similarityPairs = sorted(similarityPairs, key= itemgetter(2), reverse= True)

# Remove duplicate pairs
for pair in similarityPairs:
    currentPair = [pair[0], pair[1]]
    reversePair = [pair[1], pair[0]]
    for otherPairs in similarityPairs:
        checkPair = [otherPairs[0], otherPairs[1]]
        if checkPair == currentPair:
            continue
        elif checkPair == reversePair:
            similarityPairs.remove(otherPairs)

# Print the top 5 similar user
print("The top 5 similar users are:\n", similarityPairs[0:5])

KeyboardInterrupt: 

In [ ]:
## FOR A GIVEN USER_ID, IDENTIFY THE TOP-5 SIMILAR USERS AND HENCE SONG RECOMMENDATIONS FROM OTHER USER'S LISTS
# Finding similar users for user 0 
tempUserXSimilar = list(mostSimilarUsersTo(0))

# Changing user index to user id
userXSimilar = []
for simUser in tempUserXSimilar:
    temp = []
    for user in userSongDict:
        if simUser[0] == user:
            temp.append(userSongDict[user])
            temp.append(simUser[1])
    userXSimilar.append(temp)

# Pulls top 5 most similar users
userXSimilar = userXSimilar[0:5]

# Grabbing just names of top 5 similar users
simUserID = []
for user in userXSimilar:
    simUserID.append(user[0])

# Finding the top 5 songs recommended based on similar users
def userSuggestions(userID, includeCurrentSongs = False):
    suggestions = {}
    for otherUserID, similarity in mostSimilarUsersTo(userID):
        for song in songsWithoutUser[otherUserID]:
            suggestions[song] = suggestions[song] = similarity

    suggestions = sorted(suggestions.items(),
                        key = lambda pair: pair[1],
                        reverse= True)
                
    if includeCurrentSongs:
        return suggestions
    else:
        return [(suggestion, weight)
                for suggestion, weight in suggestions
                    if suggestion not in songsWithoutUser[userID]]

top5SongsWeights = userSuggestions(0, False)
top5SongsWeights = top5SongsWeights[0:5]

songRecommendations = []
for song in top5SongsWeights:
    songRecommendations.append(song[0])

#print(songRecommendations)

print("The top 5 users this user has matched with are:", simUserID, "\n\nThe top 5 recommended songs from similar users are:", songRecommendations)


The top 5 users this user has matched with are: ['215ead8d62a0a100c5ebf9fdb7a42d93145eb19b', '58762a4d9857daba21681a885d44ceb26eb97358', 'fbe76fedef949361315193fe3cf85f184042dc1b', 'ea4cf1fe6983bd5b939c3788ed516a10fcc1471f', 'bab630566021c8c6de68c293c21ff89ea4aeab7f'] 

The top 5 recommended songs from similar users are: ['187809', '379742', '206339', '147298', '152870']
